In [1]:
import os
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array


In [5]:
from bing_image_downloader import downloader
downloader.download("Manga Comic Strips", limit=100,  output_dir='manga_comics/', adult_filter_off=True, force_replace=False, timeout=600, verbose=True)

ModuleNotFoundError: No module named 'bing_image_downloader'

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import shutil

base_url = 'https://www.imdb.com/title/tt2560140/mediaindex?page={}&ref_=ttmi_mi_sm'
img_urls = []

def scrape_images(url):
    raw = requests.get(url)
    if raw.status_code == 200:
        soup = BeautifulSoup(raw.content, 'html.parser')
        images = soup.find_all('div', class_='media_index_thumb_list')   #gallery-image-wrapper
        if images:
            for div in images:
                img_tags = div.find_all('img')  # Find all img tags within the div
                for img_tag in img_tags:
                    img_url = img_tag.get('src')  # Extracting the image URL
                    if img_url:
                        img_urls.append(img_url)
        else:
            print("No images found on page:", url)
    else:
        print("Failed to fetch page:", url)

# Create a folder to store images
folder_path = 'aot_images'  # Change the folder name here
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for page_number in range(1, 2):  # Adjust the range according to your assumption
    page_url = base_url.format(page_number)
    print("Scraping images from page:", page_url)
    scrape_images(page_url)

# Download images
for idx, img_url in enumerate(img_urls):
    # Make a request to the image URL
    img_source = requests.get(img_url, stream=True)

    # Save image to folder_path
    with open(os.path.join(folder_path, f'img{idx}.jpg'), 'wb') as file:  # Save to the new folder
        shutil.copyfileobj(img_source.raw, file)
        print(f"Saved image: img{idx}.jpg")  # Print statement to show which image is being saved

    del img_source  # Remove the file from memory

    if idx > 30:  # Limiting to 200 images for demonstration
        break


Scraping images from page: https://www.imdb.com/title/tt2560140/mediaindex?page=1&ref_=ttmi_mi_sm
Saved image: img0.jpg
Saved image: img1.jpg
Saved image: img2.jpg
Saved image: img3.jpg
Saved image: img4.jpg
Saved image: img5.jpg
Saved image: img6.jpg
Saved image: img7.jpg
Saved image: img8.jpg
Saved image: img9.jpg
Saved image: img10.jpg
Saved image: img11.jpg
Saved image: img12.jpg
Saved image: img13.jpg
Saved image: img14.jpg
Saved image: img15.jpg
Saved image: img16.jpg
Saved image: img17.jpg
Saved image: img18.jpg
Saved image: img19.jpg
Saved image: img20.jpg
Saved image: img21.jpg
Saved image: img22.jpg
Saved image: img23.jpg
Saved image: img24.jpg
Saved image: img25.jpg
Saved image: img26.jpg
Saved image: img27.jpg
Saved image: img28.jpg
Saved image: img29.jpg
Saved image: img30.jpg
Saved image: img31.jpg


In [3]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urlparse, urljoin

# Example usage:
url = "http://pt.jikos.cz/garfield/{}/"
folder = "garfield_comics"

def download_images_from_page(url, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all <img> tags within the table
    image_tags = soup.select('table img')
    
    # Extract image URLs and download images
    for i, img_tag in enumerate(image_tags):
        # Get the value of the 'src' attribute
        img_url = img_tag.get('src')
        if img_url:
            # Construct absolute URL if the 'src' attribute is a relative URL
            parsed_url = urlparse(url)
            img_url = urljoin(parsed_url.scheme + '://' + parsed_url.netloc, img_url)
            
            # Download image
            img_name = f"{folder}_{i}.jpg"
            img_path = os.path.join(folder, img_name)
            with open(img_path, 'wb') as f:
                img_data = requests.get(img_url).content
                f.write(img_data)


for year in range(1978, 1980):  # Adjust the range according to your assumption
    year_url = url.format(year)
    print("Scraping images from page:", year_url)
    download_images_from_page(year_url, folder)

Scraping images from page: http://pt.jikos.cz/garfield/1978/
Scraping images from page: http://pt.jikos.cz/garfield/1979/


In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [34]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [35]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
2/2 [==============================] - 13s 5s/step - loss: 1.9463 - accuracy: 0.4667 - val_loss: 0.0079 - val_accuracy: 1.0000
Epoch 2/10
2/2 [==============================] - 11s 4s/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.2364e-06 - val_accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 12s 4s/step - loss: 5.4270e-07 - accuracy: 1.0000 - val_loss: 5.4674e-08 - val_accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 11s 4s/step - loss: 1.3126e-05 - accuracy: 1.0000 - val_loss: 2.2727e-07 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 11s 4s/step - loss: 4.8799e-05 - accuracy: 1.0000 - val_loss: 8.6266e-07 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 10s 4s/step - loss: 5.5230e-04 - accuracy: 1.0000 - val_loss: 1.9519e-06 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 10s 4s/step - loss: 8.6449e-04 - accuracy: 1.0000 - val_loss: 1.9591e-06 - val_accurac

In [37]:
test_loss, test_acc = model.evaluate(val_images, val_labels)
print("Test Accuracy:", test_acc)

1/1 [==============================] - 1s 1s/step - loss: 2.7796e-07 - accuracy: 1.0000
Test Accuracy: 1.0


In [43]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array

# Define the image dimensions
img_width, img_height = 224, 224

# Function to preprocess images
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = img_array.astype("float") / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# List to store image paths and corresponding labels
image_paths = []
labels = []

# Assuming you have directories named 'manga' and 'classical' containing images
manga_dir = '/Users/gulbansal/Documents/afoexam/aot_images'
classical_dir = '/Users/gulbansal/Documents/afoexam/garfield_comics'

# Iterate over manga images
for filename in os.listdir(manga_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(manga_dir, filename)
        image_paths.append(img_path)
        labels.append(0)  # Folder name as label

# Iterate over classical images
for filename in os.listdir(classical_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(classical_dir, filename)
        image_paths.append(img_path)
        labels.append(1)  # Folder name as label

# Convert labels to numpy array
labels = np.array(labels)

# Initialize lists to store preprocessed images
preprocessed_images = []

# Preprocess images
for img_path in image_paths:
    preprocessed_img = preprocess_image(img_path)
    preprocessed_images.append(preprocessed_img)

# Convert preprocessed images to numpy array
preprocessed_images = np.array(preprocessed_images)

# Split dataset into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(preprocessed_images, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

print("Data preprocessing completed.")
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation images shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Data preprocessing completed.
Train images shape: (45, 224, 224, 3)
Train labels shape: (45,)
Validation images shape: (5, 224, 224, 3)
Validation labels shape: (5,)
Test images shape: (13, 224, 224, 3)
Test labels shape: (13,)


In [60]:
# Load a random image
random_img_path = '/Users/gulbansal/Documents/afoexam/naruto.jpg'  # Replace with the path to your random image
random_img = preprocess_image(random_img_path)

# Make predictions using the trained model
predictions = model.predict(np.expand_dims(random_img, axis=0))

# Print predictions
print("Predictions:", predictions)


1/1 [==============================] - 0s 286ms/step
Predictions: [[0.76375127]]


In [57]:
# Set a threshold
threshold = 0.5

# Convert probability to binary class label using the threshold
binary_prediction = 1 if predictions[0][0] > threshold else 0

print(binary_prediction)

1


### Comic strips 

In [58]:
import requests
from bs4 import BeautifulSoup
import os
import shutil

# Define URLs to scrape
manga_url = "https://in.pinterest.com/emmaslavinhall/animemangacomic-strips/"
classical_url = "https://in.pinterest.com/wegardentoo/vintage-comic-book-covers/"

# Create directories to store images
os.makedirs("manga_images", exist_ok=True)
os.makedirs("classical_images", exist_ok=True)

# Function to download images from a URL
def download_images(url, output_dir):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    image_tags = soup.find_all("img")
    for img_tag in image_tags:
        img_url = img_tag.get("src")
        if img_url:
            img_name = os.path.basename(img_url)
            img_path = os.path.join(output_dir, img_name)
            with open(img_path, "wb") as img_file:
                img_file.write(requests.get(img_url).content)

# Download images for Manga and Classical categories
download_images(manga_url, "manga_images")
download_images(classical_url, "classical_images")